# Symmetric Matrix Vector Multiply Routines (stored in lower triangle)

This notebook walks you through how to implement $ y := A x + y $ where $ A $ is symmetric with only the lower triangular part stored.

## Getting started

We will use some functions that are part of our laff library (of which this function will become a part) as well as some routines from the FLAME API (Application Programming Interface) that allows us to write code that closely resembles how we typeset algorithms using the FLAME notation.  These functions are imported with `include` and `using` statements.

## Algorithm that takes dot products

<img src="../images/symv_l_dot.png" alt="Matrix-matrix multiplication by columns picture" width="80%">

## The `Symv_l_unb_var1!( A, x, y )` routine

This routine, given symmetric matrix $ A \in \mathbb{R}^{n \times n} $, $ x \in \mathbb{R}^n $, and $ y \in \mathbb{R}^n $, computes $ y := A x + y $.  The "_l_" in the name of the routine indicates that $ A $ is stored in the lower triangular part of the matrix.

The specific laff functions we will use are 
<ul>
<li> <code> laff.dots!( x, y, alpha ) </code> which computes $ \alpha := x^T y + \alpha $.  </li>
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [14]:
include("../flame.jl")
using .flame
include("../laff/laff.jl")
using .laff

function Symv_l_unb_var1!(A, x, y)

    ATL, ATR, 
    ABL, ABR  = flame.part_2x2(A, 
                               0, 0, "TL")

    xT, 
    xB  = flame.part_2x1(x, 
                         0, "TOP")

    yT, 
    yB  = flame.part_2x1(y, 
                         0, "TOP")

    while size(ATL, 1) < size(A, 1)

        A00,  a01,     A02,  
        a10t, alpha11, a12t, 
        A20,  a21,     A22   = flame.repart_2x2_to_3x3(ATL, ATR, 
                                                       ABL, ABR, 
                                                       1, 1, "BR")

        x0,   
        chi1, 
        x2    = flame.repart_2x1_to_3x1(xT, 
                                        xB, 
                                        1, "BOTTOM")

        y0,   
        psi1, 
        y2    = flame.repart_2x1_to_3x1(yT, 
                                        yB, 
                                        1, "BOTTOM")

        #------------------------------------------------------------#

        laff.dots!( a10t, x0, psi1 )
        laff.dots!( alpha11, chi1, psi1 )
        laff.dots!( a21, x2, psi1 )

        #------------------------------------------------------------#

        ATL, ATR, 
        ABL, ABR  = flame.cont_with_3x3_to_2x2(A00,  a01,     A02,  
                                               a10t, alpha11, a12t, 
                                               A20,  a21,     A22,  
                                               "TL")

        xT, 
        xB  = flame.cont_with_3x1_to_2x1(x0,   
                                         chi1, 
                                         x2,   
                                         "TOP")

        yT, 
        yB  = flame.cont_with_3x1_to_2x1(y0,   
                                         psi1, 
                                         y2,   
                                         "TOP")

    end

    flame.merge_2x1!(yT, 
                     yB, y)

end

Symv_l_unb_var1! (generic function with 1 method)

## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [31]:
A = rand(4, 4)
x = rand(4)
y = rand(4)
yold = rand(4)

println("A before =")
A

A before =


4×4 Array{Float64,2}:
 0.00683577  0.122392    0.898256  0.451886
 0.426623    0.822453    0.334979  0.768391
 0.38829     0.251834    0.63654   0.32121 
 0.713301    0.00696825  0.305923  0.823202

In [32]:
println("x before =")
x

x before =


4-element Array{Float64,1}:
 0.2393061314379259 
 0.6787710941357248 
 0.47578514812397477
 0.3466716941426544 

In [33]:
println("y before =")
y

y before =


4-element Array{Float64,1}:
 0.5783880432248696
 0.6425102098133013
 0.6341377016939811
 0.674441492295432 

In [34]:
using LinearAlgebra
# Notice that A is not symmetric.  We now "symmetrize it", by using the
# lower triangular part of the matrix (:L) in the upper triangular part
Asymm = Symmetric(A, :L)

laff.copy!( y, yold )   # save the original vector y

Symv_l_unb_var1!( A, x, y )

println( "y after =" )
y

y after =


4-element Array{Float64,1}:
 1.301627001670667 
 1.425095230317646 
 1.3069065185762554
 1.2808033273971056

In [35]:
println( "y - ( Asymm * x + yold ) = " )
y - ( Asymm * x + yold )

y - ( Asymm * x + yold ) = 


4-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0

Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.

## Algorithm that uses axpys

<img src="../images/symv_l_axpy.png" alt="Matrix-matrix multiplication by columns picture" width="80%">

## The `Symv_l_unb_var2!( A, x, y )` routine

This routine, given symmetric matrix $ A \in \mathbb{R}^{n \times n} $, $ x \in \mathbb{R}^n $, and $ y \in \mathbb{R}^n $, computes $ y := A x + y $.  The "_l_" in the name of the routine indicates that $ A $ is stored in the lower triangular part of the matrix.

The specific laff functions we will use are
<ul>
<li> <code> laff.axpy!( alpha, x, y ) </code> which computes $ y := \alpha x + y $.  </li>
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [20]:
include("../flame.jl")
using .flame
include("../laff/laff.jl")
using .laff

function Symv_l_unb_var2!(A, x, y)

    ATL, ATR, 
    ABL, ABR  = flame.part_2x2(A, 
                               0, 0, "TL")

    xT, 
    xB  = flame.part_2x1(x, 
                         0, "TOP")

    yT, 
    yB  = flame.part_2x1(y, 
                         0, "TOP")

    while size(ATL, 1) < size(A, 1)

        A00,  a01,     A02,  
        a10t, alpha11, a12t, 
        A20,  a21,     A22   = flame.repart_2x2_to_3x3(ATL, ATR, 
                                                       ABL, ABR, 
                                                       1, 1, "BR")

        x0,   
        chi1, 
        x2    = flame.repart_2x1_to_3x1(xT, 
                                        xB, 
                                        1, "BOTTOM")

        y0,   
        psi1, 
        y2    = flame.repart_2x1_to_3x1(yT, 
                                        yB, 
                                        1, "BOTTOM")

        #------------------------------------------------------------#

        laff.axpy!( chi1, a10t, y0 )
        laff.axpy!( chi1, alpha11, psi1 )
        laff.axpy!( chi1, a21, y2 )

        #------------------------------------------------------------#

        ATL, ATR, 
        ABL, ABR  = flame.cont_with_3x3_to_2x2(A00,  a01,     A02,  
                                               a10t, alpha11, a12t, 
                                               A20,  a21,     A22,  
                                               "TL")

        xT, 
        xB  = flame.cont_with_3x1_to_2x1(x0,   
                                         chi1, 
                                         x2,   
                                         "TOP")

        yT, 
        yB  = flame.cont_with_3x1_to_2x1(y0,   
                                         psi1, 
                                         y2,   
                                         "TOP")

    end

    flame.merge_2x1!(yT, 
                     yB, y)

end

Symv_l_unb_var2! (generic function with 1 method)

## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [36]:
A = rand(4, 4)
x = rand(4)
y = rand(4)
yold = rand(4)

println("A before =")
A

A before =


4×4 Array{Float64,2}:
 0.0201572  0.419734  0.946606  0.108028 
 0.573067   0.390595  0.865307  0.961154 
 0.852282   0.257836  0.989444  0.348767 
 0.890922   0.879576  0.448985  0.0816965

In [37]:
println("x before =")
x

x before =


4-element Array{Float64,1}:
 0.4518093664216043 
 0.6981569571595654 
 0.900357079889964  
 0.44483098875068516

In [38]:
println("y before =")
y

y before =


4-element Array{Float64,1}:
 0.9206580553003165 
 0.9549206340288459 
 0.6081200407205445 
 0.22330166108965943

In [39]:
# Notice that A is not symmetric.  We now "symmetrize it"
Asymm = Symmetric(A, :L)

laff.copy!( y, yold )   # save the original vector y

Symv_l_unb_var2!( A, x, y )

println( "y after =" )
y

y after =


4-element Array{Float64,1}:
 2.4935241489406756
 2.109941963680696 
 2.2637745203202377
 1.6804987734133805

In [40]:
println( "y - ( Asymm * x + yold ) = " )
y - ( Asymm * x + yold )

y - ( Asymm * x + yold ) = 


4-element Array{Float64,1}:
  0.0                  
  0.0                  
  4.440892098500626e-16
 -2.220446049250313e-16

Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.